https://jalammar.github.io/illustrated-gpt2/

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import * # Input, Embedding, Dense, LayerNormalization, MultiheadAttention, Dropout
from tensorflow.keras.models import Model

In [2]:
class GPT2(tf.keras.Model):
    def __init__(self, vocab_size, max_sequence_length, num_layers=12, num_heads=8, d_model=512, dff=1024, dropout_rate=0.0):
        super(GPT2, self).__init__()
        self.num_layers = num_layers
        self.d_model = d_model

        self.embedding = Embedding(vocab_size, d_model) ## 입력 문자열을 정해진 길이의 벡터로 임베딩, Simple traditional embedding
        self.positional_encoding = self.positional_encoding(max_sequence_length, d_model) ## 위치 정보 부여를 위한 positional encoding, RNN 기반 레이어와 달리, Attention 레이어에는 데이터가 병렬로 입력되기 때문.

        self.encoder_layer = [EncoderLayer(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)] ## dff is dimension of feed-forward
        self.dropout = Dropout(dropout_rate)

        self.final_layer = Dense(vocab_size)

    def call(self, inputs):
        sequence_length = tf.shape(inputs)[1]
        mask = self.create_padding_mask(inputs)

        # Embedding and Positional Encoding
        x = self.Embedding(inputs)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.positional_encoding[:, :sequence_length, :] # return [batch, sequence_length, model_dimension]
        x = self.dropout(x)

        # Encoder Layers
        for i in range(self.num_layers):
            x = self.encoder_layer[i](x, mask)

        output = self.final_layer(x)

        return output

    def positional_encoding(self, sequence_length, d_model):
        position = tf.expand_dims(tf.range(0, sequence_length, dtype=tf.float32), axis=1)
        div_term = tf.pow(10000, 2 * tf.range(0, d_model, 2, dtype=tf.float32) / d_model)
        pe = tf.concat([tf.sin(position / div_term), tf.cos(position / div_term)], axis=1)
        return tf.expand_dims(pe, axis=0)

    # 특정 단어의 마스킹을 위함
    def create_padding_mask(self, sequence):
        mask = tf.cast(tf.math.equal(sequence, 0), tf.float32)
        return mask[:, tf.newaxis, tf.newaxis, :]

In [3]:
from keras.layers.attention.multi_head_attention import MultiHeadAttention
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout_rate):
        super(EncoderLayer, self).__init__()
        # need to attention layer, dropout layer, layernorm layer and feed-forward layer
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.dropout1 = Dropout(dropout_rate)
        self.ln1 = LayerNormalization(epsilon=1e-6)

        self.ffn = self.point_wise_feed_forward_network(d_model, dff)
        self.dropout2 = Dropoit(dropout_rate)
        self.ln2 = LayerNormalization(epsilon=1e-6)

    def call(self, inputs, mask):
        attention_output = self.mha(inputs, inputs, attention_mask=mask)
        attention_output = self.dropout1(attention_output)
        output1 = self.ln1(inputs + attention_output)

        ffn_output = self.ffn(output1)
        ffn_output = self.dropout2(ffn_output)
        output2 = self.ln2(output1 + ffn_output)

        return output2

    def point_wise_feed_forward_network(self, d_model, dff):
        return tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])

Training Example

In [ ]:
num_layers = 12
num_heads = 8
d_model = 512
dff = 1024
dropout_rate = 0.1

# Create an instance of the GPT-2 model
gpt2 = GPT2(vocab_size, max_sequence_length, num_layers, num_heads, d_model, dff, dropout_rate)

# Compile the model
gpt2.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Train the model
gpt.fit(train_dataset, epochs=10)

GPT-2 with HuggingFace transformers

In [4]:
!pip install transformers

In [7]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'
model = TFGPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Generate text using the GPT-2 model
def generate_text(input_text, max_length=100):
    input_ids = tokenizer.encode(input_text, return_tensors='tf')
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Example usage:
input_sequence = 'Once upon a time'
generated_text = generate_text(input_sequence, max_length=100)

generated_text

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great'